In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Dropout
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot as plt
import seaborn as sns

DataSet = pd.read_csv('../input/electric-motor-temperature/measures_v2.csv')

In [ ]:
X = DataSet[DataSet['profile_id'] != 20].drop(columns = ['pm','profile_id'])
y = DataSet['pm'][DataSet['profile_id'] != 20]

X_test = DataSet[DataSet['profile_id'] == 20].drop(columns = ['pm','profile_id'])
y_test = DataSet['pm'][DataSet['profile_id'] == 20]

scaler1 = StandardScaler()
scaler2 = StandardScaler()
X = scaler1.fit_transform(X)
y = scaler2.fit_transform(np.asarray(y).reshape(-1, 1))

X_test = scaler1.transform(X_test)
y_test = scaler2.transform(np.asarray(y_test).reshape(-1, 1))

X_train, X_rnn, y_train, y_rnn = train_test_split(X,y, test_size = 0.5,shuffle=False)

In [ ]:
from keras.models import Model
from keras.layers import Input
from keras.layers.merge import concatenate


input_1 = Input(shape=(X.shape[1]))
hlay1_1 = Dense(16,activation = 'relu')(input_1)
hlay2_1 = Dense(16,activation = 'relu')(hlay1_1)
final_dnn1 = Dense(1,activation = 'linear')(hlay2_1)
dnn1 = Model(inputs=input_1, outputs=final_dnn1)
dnn1.compile(optimizer = "adam", loss = "mean_squared_error")

input_2 = Input(shape=(X.shape[1]))
hlay1_2 = Dense(32,activation = 'relu')(input_2)
hlay2_2 = Dense(16,activation = 'relu')(hlay1_2)
final_dnn2 = Dense(1,activation = 'linear')(hlay2_2)
dnn2 = Model(inputs=input_2, outputs=final_dnn2)
dnn2.compile(optimizer = "adam", loss = "mean_absolute_percentage_error")

input_3 = Input(shape=(X.shape[1]))
hlay1_3 = Dense(16,activation = 'sigmoid')(input_3)
hlay2_3 = Dense(16,activation = 'sigmoid')(hlay1_3)
hlay3_3 = Dense(8,activation = 'relu')(hlay2_3)
final_dnn3 = Dense(1,activation = 'linear')(hlay3_3)
dnn3 = Model(inputs=input_3, outputs=final_dnn3)
dnn3.compile(optimizer = "adam", loss = "mean_squared_error")

input_4 = Input(shape=(X.shape[1]))
hlay1_4 = Dense(32,activation = 'relu')(input_4)
hlay2_4 = Dense(32,activation = 'sigmoid')(hlay1_4)
final_dnn4 = Dense(1,activation = 'linear')(hlay2_4)
dnn4 = Model(inputs=input_4, outputs=final_dnn4)
dnn4.compile(optimizer = "adam", loss = "mean_squared_error")

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.6)
dnn1.fit(X_train, y_train, epochs = 15, batch_size = 32,verbose=True,validation_data=(X_val, y_val))

X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.6)
dnn2.fit(X_train, y_train, epochs = 15, batch_size = 32,verbose=True,validation_data=(X_val, y_val))

X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.6)
dnn3.fit(X_train, y_train, epochs = 15, batch_size = 32,verbose=True,validation_data=(X_val, y_val))

X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.6)
dnn4.fit(X_train, y_train, epochs = 15, batch_size = 64,verbose=True,validation_data=(X_val, y_val))

In [ ]:
dnn1.trainable = False; dnn2.trainable = False; dnn3.trainable = False; dnn4.trainable = False;

enn_input = Input(shape=(X_train.shape[1]))
const_1 = dnn1(enn_input)
const_2 = dnn2(enn_input)
const_3 = dnn3(enn_input)
const_4 = dnn4(enn_input)
enn_1 = Dense(8,activation = 'relu')(enn_input)
merge = concatenate(inputs = [const_1,const_2,const_3,const_4,enn_1])
enn_2 = Dense(32,activation = 'relu')(merge)
enn_3 = Dense(32,activation = 'relu')(enn_2)
final = Dense(1,activation = 'linear')(enn_3)
enn = Model(inputs=enn_input, outputs=final)
enn.compile(optimizer = "adam", loss = "mean_squared_error")

X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.2)

history = enn.fit(X_train, y_train, epochs = 100, batch_size = 32,verbose=True,validation_data=(X_val, y_val))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training History')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend(['Training','Validation'])
plt.show()

In [ ]:
y_pred = enn.predict(X_train)
plt.plot(scaler2.inverse_transform(y_train[1100:1200]))
plt.plot(scaler2.inverse_transform(y_pred[1100:1200]))
plt.legend(['True','Prediction'])
plt.title('Train Set')
plt.show()

In [ ]:
y_pred_test = enn.predict(X_test)
plt.plot(scaler2.inverse_transform(y_test))
plt.plot(scaler2.inverse_transform(y_pred_test))
plt.legend(['True','Prediction'])
plt.title('Test Set')
plt.show()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_rnn,y_rnn, test_size = 0.2)

X_train_rnn = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
y_train_rnn = np.reshape(y_train, (y_train.shape[0],1))
X_val_rnn = np.reshape(X_val, (X_val.shape[0], X_val.shape[1],1))
y_val_rnn = np.reshape(y_val, (y_val.shape[0],1))

enn.trainable = False;

input_rnn = Input(shape=(X_train.shape[1], 1))
input_enn = Input(shape=(X_train.shape[1]))
rnn1 = SimpleRNN(16, return_sequences=True)(input_rnn)
drop1 = Dropout(0.2)(rnn1)
rnn2 = SimpleRNN(16, return_sequences=False)(drop1)
drop2 = Dropout(0.2)(rnn2)
enn_lay = enn(input_enn)
merge = concatenate(inputs = [enn_lay,drop2])
combine1 = Dense(16,activation = 'relu')(merge)
combine2 = Dense(16,activation = 'relu')(combine1)
final = Dense(1)(combine2)

ernn = Model(inputs=[input_rnn,input_enn], outputs=final)
ernn.compile(optimizer = "adam", loss = "mean_squared_error")
plot_model(ernn)
history = ernn.fit([X_train_rnn,X_train], y_train, epochs = 50, batch_size = 32,verbose=True,validation_data=([X_val_rnn,X_val], y_val))


In [ ]:
history = ernn.fit([X_train_rnn,X_train], y_train, epochs = 50, batch_size = 32,verbose=True,validation_data=([X_val_rnn,X_val], y_val))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training History')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend(['Training','Validation'])
plt.show()

In [ ]:
y_pred = ernn.predict([X_train_rnn,X_train])
plt.plot(scaler2.inverse_transform(y_train[1100:1200]))
plt.plot(scaler2.inverse_transform(y_pred[1100:1200]))
plt.legend(['True','Prediction'])
plt.title('Train Set')
plt.show()

In [ ]:
X_test_rnn = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

y_pred_test = ernn.predict([X_test_rnn,X_test])
plt.plot(scaler2.inverse_transform(y_test))
plt.plot(scaler2.inverse_transform(y_pred_test))
plt.legend(['True','Prediction'])
plt.title('Test Set')
plt.show()